In [2]:
# Automatically reload modules when they change
%load_ext autoreload
%autoreload 2  

# Import necessary libraries and load environment variables
import os
from dotenv import load_dotenv
load_dotenv(override=True)  # Load environment variables from .env file (contains API keys for OpenAI/Azure OpenAI)

# Import data modeling and typing libraries
from pydantic import BaseModel
from typing import List, Dict, Union
from typing import Any
from pathlib import Path
import re
import json
from typing_extensions import Literal

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.contents import ChatHistory, StreamingChatMessageContent
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.agents import Agent, ChatCompletionAgent
from semantic_kernel.agents import ChatHistoryAgentThread
from semantic_kernel.functions.kernel_arguments import KernelArguments

from plugins import *

from rich.console import Console
console = Console()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✅ All utility plugins defined!
🔧 Available plugins: Weather, Random, DateTime, TextAnalysis, RAG, NL2SQL, Cosmos


## Defining Agent Hierarchy

### Agent Hierarchy Visualization


```
┌────────────────────────────┐
│     Main Orchestration     │
│     (SelectorAgent)        │
└───────────┬────────────────┘
            │
    ┌─────────────────┼
    │                 │ 
┌───▼────┐    ┌──────▼──────────────────┐
│  RAG   │    │  CombinedDatabaseAgent  │
│ Agent  │    │        (Local)          │
│(Local) │    └───────────┬─────────────┘
└───┬────┘                │
    │           ┌─────────┼─────────┐
┌───▼────┐      │                   │
│  RAG   │  ┌───▼────┐          ┌─▼─────┐
│Plugin  │  │Cosmos  │          │SQL    │
│(Local) │  │Agent   │          │Agent  │
└────────┘  │(Local) │          │(Local)│
            └───┬────┘          └───┬───┘
                │                   │
            ┌───▼────┐          ┌───▼────┐
            │Cosmos  │          │NL2SQL  │
            │Plugin  │          │Plugin  │
            │(Local) │          │(Local) │
            └────────┘          └────────┘
```



In [18]:
cosmos_system_message = """
You are a Cosmos plugin that simulates retrieving multimodal documents in JSON format about last year's sales analysis of a company that sells smart city IoT solutions.
You will return a context that contains the relevant information from the multimodal documents based on the user's query.
"""

cosmos_agent = ChatCompletionAgent(
    id="CosmosIoTSalesAgent",
    name="CosmosIoTSalesAgent",
    instructions=cosmos_system_message,
    description="Router Agent for Cosmos Plugin. This agent simulates retrieving multimodal documents in JSON format about last year's sales analysis of a company that sells smart city IoT solutions.",
    plugins=[cosmos_plugin],
    service=AzureChatCompletion()
)


nl2sql_system_message = """You are an NL2SQL plugin that simulates retrieving search results from a SQL server related to personal banking transactions.
You will return a context that contains the relevant information from the SQL server based on the user's query.
"""

sql_agent = ChatCompletionAgent(
    id="NL2SQLAgent",
    name="NL2SQLAgent",
    instructions=nl2sql_system_message,
    description="Router Agent for NL2SQL Plugin. This agent retrieves search results from a SQL server related to personal banking transactions and returns it as context.",
    plugins=[nl2sql_plugin],
    service=AzureChatCompletion()
)


rag_system_message = """You are a RAG plugin that simulates retrieving search results from a data source related to a Car Insurance Policy of a BMW 320i.
You will return a context that contains the relevant information from the data source based on the user's query.
"""

rag_agent = ChatCompletionAgent(
    id="RAGAgent",
    name="RAGAgent",
    instructions=rag_system_message,
    description="Router Agent for RAG Plugin. This agent retrieves search results from a data source related to a Car Insurance Policy of a BMW 320i and returns it as context.",
    plugins=[rag_plugin],
    service=AzureChatCompletion()
)


# Combined Database Agent
combined_database_system_message = """You are a Combined Database Agent that simulates retrieving search results from multiple data sources.
You will return a context that contains the relevant information from the data sources based on the user's query.
You will use the NL2SQL plugin for personal banking transactions, and the Cosmos plugin for multimodal documents related to sales analysis of smart city IoT solutions.
"""

combined_database_agent = ChatCompletionAgent(
    id="CombinedDatabaseAgent",
    name="CombinedDatabaseAgent",
    instructions=combined_database_system_message,
    description="Router Agent for Combined Database. This agent retrieves search results from multiple data sources including NL2SQL and Cosmos agents.",
    plugins=[cosmos_agent, sql_agent],
    service=AzureChatCompletion()
)


# Combined Data Sources Agent
combined_data_sources_system_message = """You are a Combined Data Sources Agent that simulates retrieving search results from multiple data sources.
You will return a context that contains the relevant information frm the data sources based on the user's query.
You will use the RAG plugin for Car Insurance Policy of a BMW 320i, the NL2SQL plugin for personal banking transactions, and the Cosmos plugin for multimodal documents related to sales analysis of smart city IoT solutions.
"""

combined_data_sources_agent = ChatCompletionAgent(
    id="CombinedDataSourcesAgent",
    name="CombinedDataSourcesAgent",
    instructions=combined_data_sources_system_message,
    description="Router Agent for Combined Data Sources. This agent retrieves search results from multiple data sources including RAG, NL2SQL, and Cosmos agents.",
    plugins=[rag_agent, combined_database_agent],
    service=AzureChatCompletion()
)


## Method 1: Agent Routing with Structured Outputs

In [23]:
kernel = Kernel()

class AgentSelection(BaseModel):
    selected_agent: str

system_message = """
You are a helpful agent router. Please select the most appropriate agent to do the task.
Available agents are:
- RAGAgent: For retrieving context from a data source about a Car Insurance Policy of a BMW 320i.
- CosmosAgent: For retrieving multimodal documents about last year's sales analysis of a company that sells smart city IoT solutions.
- SQLAgent: For retrieving search results from a SQL server related to personal banking transactions.
You will receive a task description, and you should return the name of the agent that is best suited to handle the task.
The output should be a JSON object with a single key "selected_agent" and the value being the name of the selected agent.
"""

settings = AzureChatPromptExecutionSettings()
settings.response_format = AgentSelection

router_agent = ChatCompletionAgent(
    id="RouterAgent",
    name="RouterAgent",
    instructions=system_message,
    description="This agent routes tasks to the appropriate data source agent.",
    service=AzureChatCompletion(),
    arguments=KernelArguments(settings=settings),
)

# Example queries to test routing
# query = "What are my last 3 transactions?"
# query = "Are my tyres insured?"
query = "What was the total sales in USD of the Q4 in the last year?"

async for response in router_agent.invoke(messages=query):                
    content = response.content
    answer = AgentSelection.model_validate(json.loads(str(response.content)))

console.print(f"Router Agent selected: {answer.selected_agent}")

if answer.selected_agent == "RAGAgent":
    # Call RAGAgent
    pass 
elif answer.selected_agent == "CosmosAgent":
    # Call CosmosAgent
    pass
elif answer.selected_agent == "SQLAgent":
    # Call SQLAgent
    pass
else:
    console.print(f"Unknown agent selected: {answer.selected_agent}")
    raise ValueError(f"Unknown agent selected: {answer.selected_agent}")




Router Agent selected: CosmosAgent

## Method 2: Agent Routing with Plugins



In [ ]:
selector_agent_system_message = f"""You are a helpful agent selector. Please select the most appropriate agent to do the task.
The selected agent should be one of the following:
- RAGAgent: For retrieving context from a data source about a Car Insurance Policy of a BMW 320i.
- CombinedDatabaseAgent: For retrieving context from multiple database sources including SQL server for personal banking transactions and Cosmos server for multimodal documents about last year's sales analysis of a company that sells smart city IoT solutions.
You will receive a task description, and you should call the proper agent to get the final answer. Please mention in your final answer the name of the agent you called.
- RandomPlugin: For generating a random number between min and max values.
"""


selector_agent = ChatCompletionAgent(
    id="SelectorAgent",
    name="SelectorAgent",
    instructions=selector_agent_system_message,
    description="This agent generates images based on the user's request.",
    plugins=[combined_database_agent, rag_agent, RandomPlugin()],
    service=AzureChatCompletion(),
)

query = "What are my last 3 transactions?"
query = "are my tyres insured?"
query = "What was the total sales in USD of the Q4 in the last year?"
query = "Give me a random nnumber between 88 and 120"

async for response in selector_agent.invoke(messages=query):                
    # thread = response.thread                    
    content = response.content


console.print(f"Agent Response:\n{content}")

🎲 Generated random number: 109 (between 88 and 120)


Agent Response:
The random number between 88 and 120 is: 109. (Generated by the RandomPlugin agent.)